In [1]:
# code to drive mount

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pyspark
!pip install lightfm
!pip install gensim
!pip install textblob
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=804892 sha256=5aa7e6f459405f7ac29250a9301b0977848dde1314fbe091729f32bec4d19a9b
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, BooleanType, DateType
from pyspark.sql.functions import to_date
import pandas as pd
import os
from pyspark.sql.functions import udf, col
from pyspark.sql.types import DoubleType


In [4]:
import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report

In [5]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [6]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [7]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Filter CSV Data") \
    .getOrCreate()

# Load the CSV file into a DataFrame
# review_df = spark.read.csv("/content/drive/MyDrive/ProcessedCSV/final_review.csv", header=True, inferSchema=True)

review_df = spark.read.csv("/content/drive/MyDrive/ProcessedCSV/Recommender_System_Newdata.csv", header=True, inferSchema=True)

# Get the column names
columns_to_drop = review_df.columns

# Filter rows where column values do not match the first row
# review_df = review_df.filter(F.col(columns_to_drop[0]) != review_df.first()[columns_to_drop[0]])
# for col in columns_to_drop[1:]:
#     review_df = review_df.filter(F.col(col) != review_df.first()[col])

# Show the filtered DataFrame
review_df.show()


+--------------------+--------------------+--------------------+--------------------+--------------+-----+-----------+-------------+---------------+------+------------+--------------------+-------+----+------------+
|         business_id|            rev_text|            bus_name|             address|          city|state|postal_code|     latitude|      longitude|rating|review_count|            tip_text|parking|WiFi|cuisine_type|
+--------------------+--------------------+--------------------+--------------------+--------------+-----+-----------+-------------+---------------+------+------------+--------------------+-------+----+------------+
|zljfjg_zTL3ceGi_V...|It's ok if you li...|Sheraton New Orle...|        500 Canal St|   New Orleans|   LA|      70130|29.9521458122| -90.0681066513|   3.0|         673|                NULL|    Yes| Yes|      Creole|
|3Y2PxEwN2iNHAQj4Z...|"Love their smoot...|       Smoothie King|    1413 S Hanley Rd|     Brentwood|   MO|      63144|    38.627224| -90

In [8]:
selected_df = review_df.select("business_id", "rev_text")

selected_df.show()

+--------------------+--------------------+
|         business_id|            rev_text|
+--------------------+--------------------+
|zljfjg_zTL3ceGi_V...|It's ok if you li...|
|3Y2PxEwN2iNHAQj4Z...|"Love their smoot...|
|TdyHRkxn334JxtUSd...|A breakfast desti...|
|1l4-uhraQzNiRjppK...|Such a sweet spot...|
|hiP3H0wW45I0jZI7K...|Spicy Memory came...|
|bErZNO33F5Bb2Gtdm...|NOT as dog friend...|
|QyGPQg1-dIGAwPaBK...|The wife and I ca...|
|omKVwCFLQYs_ySD1b...|We ate here tonig...|
|eMiN8nm70jjKg8izi...|There is a bunch ...|
|hJhqAfuv2n6W6DUwh...|I feel like I hav...|
|fxZtYRBOvoddW7k3f...|"Burger was worth...|
|IWHdx0NhDKADkGOgX...|The more sandwich...|
|iDrmpQwGAAmftLzWq...|Hands down the be...|
|jL3R9dm_1sLRCAdIN...|Absolutely love t...|
|qLXO7VFOlWo2iocQB...|I gave them 1 sta...|
|tMsQnjwF4T8_a_Jbf...|Good food!! After...|
|CmmmUb9s0k0OPb6wA...|02's on Whyte is ...|
|CmmmUb9s0k0OPb6wA...|02's on Whyte is ...|
|0oSSjekU-3GR8gsel...|The new biscuits ...|
|Ssur8TPCIZWnZeCDQ...|My spouse 

In [10]:
# # prompt: concatenate all text associated with single business id to one sinlge text

# from pyspark.sql.functions import collect_list, concat_ws

# # Group by business_id and concatenate all text values
# concatenated_df = selected_df.groupBy("business_id").agg(concat_ws(" ", collect_list("rev_text")).alias("concatenated_text"))

# concatenated_df.show(truncate=False)

In [11]:
# selected_df = concatenated_df

In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Preprocess Text") \
    .getOrCreate()

# Preprocessing function
def preprocess_text(text):
    import nltk
    from nltk.corpus import stopwords
    from nltk.tokenize import word_tokenize
    from nltk.stem import WordNetLemmatizer

    # Ensure NLTK resources are downloaded on executors
    nltk.download('stopwords')
    nltk.download('punkt')
    nltk.download('wordnet')
    nltk.download('punkt_tab')

    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()

    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalpha() and word not in stop_words]
    return ' '.join(tokens)

# Register the preprocessing function as a UDF
preprocess_udf = udf(preprocess_text, StringType())

# Load your DataFrame
# review_df = spark.read.csv("/content/drive/MyDrive/ProcessedCSV/final_review.csv", header=True, inferSchema=True)

# Select only the relevant columns
# selected_df = review_df.select("business_id", "text")

# Apply the UDF to preprocess the 'text' column
preprocessed_df = selected_df.withColumn("Review", preprocess_udf(selected_df["rev_text"]))

# Show the resulting DataFrame
preprocessed_df.show(truncate=False)


+----------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [10]:
preprocessed_df = preprocessed_df.select("business_id","Review")
preprocessed_df.show()

+--------------------+--------------------+
|         business_id|              Review|
+--------------------+--------------------+
|zljfjg_zTL3ceGi_V...|ok like cockroach...|
|3Y2PxEwN2iNHAQj4Z...|love smoothy cinn...|
|TdyHRkxn334JxtUSd...|breakfast destina...|
|1l4-uhraQzNiRjppK...|sweet spot servic...|
|hiP3H0wW45I0jZI7K...|spicy memory came...|
|bErZNO33F5Bb2Gtdm...|dog friendly want...|
|QyGPQg1-dIGAwPaBK...|wife came valenti...|
|omKVwCFLQYs_ySD1b...|ate tonight first...|
|eMiN8nm70jjKg8izi...|bunch hype place ...|
|hJhqAfuv2n6W6DUwh...|feel like come de...|
|fxZtYRBOvoddW7k3f...|burger worth wort...|
|IWHdx0NhDKADkGOgX...|sandwich philly s...|
|iDrmpQwGAAmftLzWq...|hand best happy h...|
|jL3R9dm_1sLRCAdIN...|absolutely love p...|
|qLXO7VFOlWo2iocQB...|gave star could m...|
|tMsQnjwF4T8_a_Jbf...|good food bully c...|
|CmmmUb9s0k0OPb6wA...|whyte newer resta...|
|CmmmUb9s0k0OPb6wA...|whyte newer resta...|
|0oSSjekU-3GR8gsel...|new biscuit steak...|
|Ssur8TPCIZWnZeCDQ...|spouse bir

In [14]:
# """"from pyspark.sql import Window
# from pyspark.sql.functions import row_number

# # Select specific columns
# preprocessed_df = preprocessed_df.select("user_id", "Review")

# # Define a window specification to assign row numbers
# window_spec = Window.orderBy("user_id")  # You can replace "user_id" with a relevant column

# # Add a row number column
# preprocessed_df = preprocessed_df.withColumn("row_num", row_number().over(window_spec))

# # Filter out the first row
# preprocessed_df = preprocessed_df.filter(preprocessed_df.row_num != 1).drop("row_num")
# """"



SyntaxError: unterminated string literal (detected at line 15) (<ipython-input-14-b7892f0e4f4e>, line 15)

In [11]:
from pyspark.sql.functions import udf, col, when
from pyspark.sql.types import StringType

# Define the preprocessing function (already registered as UDF above)
preprocess_udf = udf(preprocess_text, StringType())

# Apply preprocessing to create 'cleaned_review'
preprocessed_df = preprocessed_df.withColumn("cleaned_review", preprocess_udf(col("Review")))

# Define sentiment and topic classification using conditions
preprocessed_df = preprocessed_df.withColumn(
    "sentiment",
    when(col("Review").rlike("(?i)good|great|excellent|incredible|fantastic|amaz"), "positive").otherwise("negative")
)

preprocessed_df = preprocessed_df.withColumn(
    "topic",
    when(col("Review").rlike("(?i)service"), "customer_service").otherwise("food_quality")
)

# Show updated DataFrame
preprocessed_df.show()


+--------------------+--------------------+--------------------+---------+----------------+
|         business_id|              Review|      cleaned_review|sentiment|           topic|
+--------------------+--------------------+--------------------+---------+----------------+
|zljfjg_zTL3ceGi_V...|ok like cockroach...|ok like cockroach...| negative|    food_quality|
|3Y2PxEwN2iNHAQj4Z...|love smoothy cinn...|love smoothy cinn...| negative|    food_quality|
|TdyHRkxn334JxtUSd...|breakfast destina...|breakfast destina...| negative|    food_quality|
|1l4-uhraQzNiRjppK...|sweet spot servic...|sweet spot servic...| positive|customer_service|
|hiP3H0wW45I0jZI7K...|spicy memory came...|spicy memory came...| negative|    food_quality|
|bErZNO33F5Bb2Gtdm...|dog friendly want...|dog friendly want...| negative|    food_quality|
|QyGPQg1-dIGAwPaBK...|wife came valenti...|wife came valenti...| negative|    food_quality|
|omKVwCFLQYs_ySD1b...|ate tonight first...|ate tonight first...| positive|    fo

In [16]:
# # prompt: group above by business id and show topics and business id

# from pyspark.sql.functions import when, col

# # Group by business_id and collect topics
# grouped_df = preprocessed_df.groupBy("business_id").agg(
#     collect_list("topic").alias("topics")
# )

# # Show the grouped DataFrame
# grouped_df.show(truncate=False)

**Collecting sentiments and topics**

In [12]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, concat_ws
from pyspark.sql.types import StringType

# Initialize SparkSession
spark = SparkSession.builder.appName("Sentiment and Topic Classification").getOrCreate()

preprocess_udf = udf(preprocess_text, StringType())

# Apply preprocessing to create 'cleaned_review'
preprocessed_df = preprocessed_df.withColumn("cleaned_review", preprocess_udf(col("Review")))

# Define the preprocessing function
def preprocess_text(text):
    return text.lower()

# Register the preprocessing function as a UDF
preprocess_udf = udf(preprocess_text, StringType())

# Apply preprocessing to create 'cleaned_review'
preprocessed_df = preprocessed_df.withColumn("cleaned_review", preprocess_udf(col("Review")))

# Define sentiment classification
preprocessed_df = preprocessed_df.withColumn(
    "sentiment",
    when(col("Review").rlike("(?i)good|great|excellent|incredible|fantastic|amazing|loved|delicious"), "positive")
    .otherwise("negative")
)

# Function to extract multiple topics
def extract_topics(review):
    topics = []
    if "service" in review or "staff" in review:
        topics.append("customer_service")
    if any(keyword in review for keyword in ["food", "taste", "delicious", "meal"]):
        topics.append("food_quality")
    if any(keyword in review for keyword in ["atmosphere", "ambiance", "decor", "music"]):
        topics.append("ambience")
    if any(keyword in review for keyword in ["price", "expensive", "cheap", "cost", "value"]):
        topics.append("price")
    return ', '.join(topics)  # Join topics into a comma-separated string

# Register the extraction function as a UDF
extract_topics_udf = udf(extract_topics, StringType())

# Apply the UDF to extract topics
preprocessed_df = preprocessed_df.withColumn("topic", extract_topics_udf(col("cleaned_review")))

# Show updated DataFrame
preprocessed_df.show()

+--------------------+--------------------+--------------------+---------+--------------------+
|         business_id|              Review|      cleaned_review|sentiment|               topic|
+--------------------+--------------------+--------------------+---------+--------------------+
|zljfjg_zTL3ceGi_V...|ok like cockroach...|ok like cockroach...| negative|                    |
|3Y2PxEwN2iNHAQj4Z...|love smoothy cinn...|love smoothy cinn...| negative|                    |
|TdyHRkxn334JxtUSd...|breakfast destina...|breakfast destina...| negative|        food_quality|
|1l4-uhraQzNiRjppK...|sweet spot servic...|sweet spot servic...| positive|customer_service,...|
|hiP3H0wW45I0jZI7K...|spicy memory came...|spicy memory came...| positive|        food_quality|
|bErZNO33F5Bb2Gtdm...|dog friendly want...|dog friendly want...| negative|                    |
|QyGPQg1-dIGAwPaBK...|wife came valenti...|wife came valenti...| positive| food_quality, price|
|omKVwCFLQYs_ySD1b...|ate tonight first.

In [13]:
# # prompt: write this df as a csv in /content/drive/MyDrive/JTG_TEST spark cscv

# # Assuming preprocessed_df is your DataFrame

# # Write the DataFrame to a CSV file in Google Drive
# preprocessed_df.write.csv("/content/drive/MyDrive/JTG_TEST/multiple_topics", header=True, mode="overwrite")

Exploding topics column

In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, when, explode
from pyspark.sql.types import StringType, ArrayType

# Initialize SparkSession
spark = SparkSession.builder.appName("Sentiment and Topic Classification").getOrCreate()

# Define the preprocessing function
def preprocess_text(text):
    return text.lower() if text else ""

# Register the preprocessing function as a UDF
preprocess_udf = udf(preprocess_text, StringType())

# Assuming preprocessed_df is already created and contains a "Review" column
preprocessed_df = preprocessed_df.withColumn("cleaned_review", preprocess_udf(col("Review")))

# Define sentiment classification
preprocessed_df = preprocessed_df.withColumn(
    "sentiment",
    when(col("Review").rlike("(?i)good|great|excellent|incredible|fantastic|amazing|loved|delicious"), "positive")
    .otherwise("negative")
)

# Function to extract multiple topics as a list
def extract_topics(review):
    topics = []
    if "service" in review or "staff" in review:
        topics.append("customer_service")
    if any(keyword in review for keyword in ["food", "taste", "delicious", "meal"]):
        topics.append("food_quality")
    if any(keyword in review for keyword in ["atmosphere", "ambiance", "decor", "music"]):
        topics.append("ambience")
    if any(keyword in review for keyword in ["price", "expensive", "cheap", "cost", "value"]):
        topics.append("price")
    return topics  # Return a list of topics

# Register the extraction function as a UDF
extract_topics_udf = udf(extract_topics, ArrayType(StringType()))

# Apply the UDF to extract topics
preprocessed_df = preprocessed_df.withColumn("topics", extract_topics_udf(col("cleaned_review")))

# Explode the topics into separate rows
exploded_df = preprocessed_df.withColumn("topic", explode(col("topics")))

# Show updated DataFrame
exploded_df.show()

+--------------------+--------------------+--------------------+---------+----------------+--------------------+
|         business_id|              Review|      cleaned_review|sentiment|           topic|              topics|
+--------------------+--------------------+--------------------+---------+----------------+--------------------+
|TdyHRkxn334JxtUSd...|breakfast destina...|breakfast destina...| negative|    food_quality|      [food_quality]|
|1l4-uhraQzNiRjppK...|sweet spot servic...|sweet spot servic...| positive|customer_service|[customer_service...|
|1l4-uhraQzNiRjppK...|sweet spot servic...|sweet spot servic...| positive|    food_quality|[customer_service...|
|hiP3H0wW45I0jZI7K...|spicy memory came...|spicy memory came...| positive|    food_quality|      [food_quality]|
|QyGPQg1-dIGAwPaBK...|wife came valenti...|wife came valenti...| positive|    food_quality|[food_quality, pr...|
|QyGPQg1-dIGAwPaBK...|wife came valenti...|wife came valenti...| positive|           price|[food

In [ ]:
# TESTING JTG JTG

In [35]:
# # Check unique counts for all categorical features
# categorical_columns = ['sentiment', 'topic', 'business_id']  # Replace with your actual categorical feature names

# for col_name in categorical_columns:
#     unique_count = train_sentiment.select(col_name).distinct().count()
#     print(f"Unique values in {col_name}: {unique_count}")

Unique values in sentiment: 2
Unique values in topic: 4
Unique values in business_id: 61


In [36]:
# from pyspark.sql import SparkSession
# from pyspark.sql.functions import expr
# from pyspark.ml.feature import StringIndexer, VectorAssembler
# from pyspark.ml.classification import RandomForestClassifier
# from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# # Initialize Spark session
# spark = SparkSession.builder.appName("Sentiment and Topic Classification").getOrCreate()


# # Step 1: Aggregate the Data
# aggregated_df = exploded_df.groupBy('business_id').agg(
#     expr("mode(sentiment) as sentiment"),
#     expr("mode(topic) as topic")
# )

# aggregated_df.show()

# # Step 2: Prepare Features
# # Encode business_id
# indexer_business = StringIndexer(inputCol='business_id', outputCol='business_id_indexed')
# aggregated_df = indexer_business.fit(aggregated_df).transform(aggregated_df)

# # Encode sentiment and topic
# indexer_sentiment = StringIndexer(inputCol='sentiment', outputCol='sentiment_indexed')
# aggregated_df = indexer_sentiment.fit(aggregated_df).transform(aggregated_df)

# indexer_topic = StringIndexer(inputCol='topic', outputCol='topic_indexed')
# aggregated_df = indexer_topic.fit(aggregated_df).transform(aggregated_df)

# # Step 3: Create feature vectors
# vector_assembler = VectorAssembler(inputCols=['business_id_indexed'], outputCol='features')
# aggregated_df = vector_assembler.transform(aggregated_df)

# # Features and target variables
# X = aggregated_df.select('features')
# y_sentiment = aggregated_df.select('sentiment_indexed')
# y_topic = aggregated_df.select('topic_indexed')

# # Step 4: Train Models
# # Split the data for sentiment prediction
# train_sentiment, test_sentiment = aggregated_df.randomSplit([0.8, 0.2], seed=42)

# # Train sentiment model with increased maxBins
# sentiment_model = RandomForestClassifier(featuresCol='features', labelCol='sentiment_indexed', maxBins=1000)
# sentiment_model = sentiment_model.fit(train_sentiment)

# # Evaluate sentiment model
# predictions_sentiment = sentiment_model.transform(test_sentiment)
# evaluator_sentiment = MulticlassClassificationEvaluator(labelCol='sentiment_indexed', predictionCol='prediction', metricName='accuracy')
# accuracy_sentiment = evaluator_sentiment.evaluate(predictions_sentiment)
# print(f"Sentiment Model Accuracy: {accuracy_sentiment}")

# # Split the data for topic prediction
# train_topic, test_topic = aggregated_df.randomSplit([0.8, 0.2], seed=42)

# # Train topic model with increased maxBins
# topic_model = RandomForestClassifier(featuresCol='features', labelCol='topic_indexed', maxBins=100)
# topic_model = topic_model.fit(train_topic)

# # Evaluate topic model
# predictions_topic = topic_model.transform(test_topic)
# evaluator_topic = MulticlassClassificationEvaluator(labelCol='topic_indexed', predictionCol='prediction', metricName='accuracy')
# accuracy_topic = evaluator_topic.evaluate(predictions_topic)
# print(f"Topic Model Accuracy: {accuracy_topic}")



+--------------------+---------+----------------+
|         business_id|sentiment|           topic|
+--------------------+---------+----------------+
|TdyHRkxn334JxtUSd...| negative|    food_quality|
|1l4-uhraQzNiRjppK...| positive|customer_service|
|hiP3H0wW45I0jZI7K...| positive|    food_quality|
|QyGPQg1-dIGAwPaBK...| positive|    food_quality|
|omKVwCFLQYs_ySD1b...| positive|    food_quality|
|eMiN8nm70jjKg8izi...| positive|    food_quality|
|fxZtYRBOvoddW7k3f...| positive|    food_quality|
|IWHdx0NhDKADkGOgX...| negative|           price|
|iDrmpQwGAAmftLzWq...| negative|    food_quality|
|jL3R9dm_1sLRCAdIN...| positive|customer_service|
|qLXO7VFOlWo2iocQB...| negative|    food_quality|
|tMsQnjwF4T8_a_Jbf...| positive|customer_service|
|CmmmUb9s0k0OPb6wA...| positive|customer_service|
|0oSSjekU-3GR8gsel...| positive|customer_service|
|Ssur8TPCIZWnZeCDQ...| positive|customer_service|
|ps1Dduk_Oqrr_O2no...| negative|customer_service|
|q3k9lwaHNqbtiGc-X...| positive|customer_service|


IllegalArgumentException: requirement failed: DecisionTree requires maxBins (= 61) to be at least as large as the number of values in each categorical feature, but categorical feature 0 has 78 values. Consider removing this and other categorical features with a large number of values, or add more training examples.

trying logistic regression Working

In [37]:
# from pyspark.sql import SparkSession
# from pyspark.sql.functions import expr
# from pyspark.ml.feature import StringIndexer, VectorAssembler
# from pyspark.ml.classification import LogisticRegression
# from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# # Initialize Spark session
# spark = SparkSession.builder.appName("Sentiment and Topic Classification").getOrCreate()

# # Step 1: Aggregate the Data
# aggregated_df = exploded_df.groupBy('business_id').agg(
#     expr("mode(sentiment) as sentiment"),
#     expr("mode(topic) as topic")
# )

# aggregated_df.show()

# # Step 2: Prepare Features
# # Encode business_id
# indexer_business = StringIndexer(inputCol='business_id', outputCol='business_id_indexed')
# aggregated_df = indexer_business.fit(aggregated_df).transform(aggregated_df)

# # Encode sentiment and topic
# indexer_sentiment = StringIndexer(inputCol='sentiment', outputCol='sentiment_indexed')
# aggregated_df = indexer_sentiment.fit(aggregated_df).transform(aggregated_df)

# indexer_topic = StringIndexer(inputCol='topic', outputCol='topic_indexed')
# aggregated_df = indexer_topic.fit(aggregated_df).transform(aggregated_df)

# # Step 3: Create feature vectors
# vector_assembler = VectorAssembler(inputCols=['business_id_indexed'], outputCol='features')
# aggregated_df = vector_assembler.transform(aggregated_df)

# # Features and target variables
# X = aggregated_df.select('features')
# y_sentiment = aggregated_df.select('sentiment_indexed')
# y_topic = aggregated_df.select('topic_indexed')

# # Step 4: Train Models
# # Split the data for sentiment prediction
# train_sentiment, test_sentiment = aggregated_df.randomSplit([0.8, 0.2], seed=42)

# # Train sentiment model using Logistic Regression
# sentiment_model = LogisticRegression(featuresCol='features', labelCol='sentiment_indexed')
# sentiment_model = sentiment_model.fit(train_sentiment)

# # Evaluate sentiment model
# predictions_sentiment = sentiment_model.transform(test_sentiment)
# evaluator_sentiment = MulticlassClassificationEvaluator(labelCol='sentiment_indexed', predictionCol='prediction', metricName='accuracy')
# accuracy_sentiment = evaluator_sentiment.evaluate(predictions_sentiment)
# print(f"Sentiment Model Accuracy: {accuracy_sentiment}")

# # Split the data for topic prediction
# train_topic, test_topic = aggregated_df.randomSplit([0.8, 0.2], seed=42)

# # Train topic model using Logistic Regression
# topic_model = LogisticRegression(featuresCol='features', labelCol='topic_indexed')
# topic_model = topic_model.fit(train_topic)

# # Evaluate topic model
# predictions_topic = topic_model.transform(test_topic)
# evaluator_topic = MulticlassClassificationEvaluator(labelCol='topic_indexed', predictionCol='prediction', metricName='accuracy')
# accuracy_topic = evaluator_topic.evaluate(predictions_topic)
# print(f"Topic Model Accuracy: {accuracy_topic}")

+--------------------+---------+----------------+
|         business_id|sentiment|           topic|
+--------------------+---------+----------------+
|TdyHRkxn334JxtUSd...| negative|    food_quality|
|1l4-uhraQzNiRjppK...| positive|customer_service|
|hiP3H0wW45I0jZI7K...| positive|    food_quality|
|QyGPQg1-dIGAwPaBK...| positive|    food_quality|
|omKVwCFLQYs_ySD1b...| positive|    food_quality|
|eMiN8nm70jjKg8izi...| positive|    food_quality|
|fxZtYRBOvoddW7k3f...| positive|    food_quality|
|IWHdx0NhDKADkGOgX...| negative|           price|
|iDrmpQwGAAmftLzWq...| negative|    food_quality|
|jL3R9dm_1sLRCAdIN...| positive|customer_service|
|qLXO7VFOlWo2iocQB...| negative|    food_quality|
|tMsQnjwF4T8_a_Jbf...| positive|customer_service|
|CmmmUb9s0k0OPb6wA...| positive|customer_service|
|0oSSjekU-3GR8gsel...| positive|customer_service|
|Ssur8TPCIZWnZeCDQ...| positive|customer_service|
|ps1Dduk_Oqrr_O2no...| negative|customer_service|
|q3k9lwaHNqbtiGc-X...| positive|customer_service|


In [43]:
# prompt: print me 5 of the businessids in a list

from pyspark.sql.functions import col

# Assuming 'preprocessed_df' is your DataFrame and it has a 'business_id' column
business_ids = preprocessed_df.select("business_id").distinct().limit(5).rdd.flatMap(lambda x: x).collect()

business_ids

['zljfjg_zTL3ceGi_Vr4OFw',
 '3Y2PxEwN2iNHAQj4ZFiGhQ',
 'TdyHRkxn334JxtUSdGpIiw',
 '1l4-uhraQzNiRjppKyMUUQ',
 'hiP3H0wW45I0jZI7KafzZg']

**Managing String indexer**

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Initialize Spark session
spark = SparkSession.builder.appName("Sentiment and Topic Classification").getOrCreate()

# Step 1: Aggregate the Data
aggregated_df = exploded_df.groupBy('business_id').agg(
    expr("mode(sentiment) as sentiment"),
    expr("mode(topic) as topic")
)

aggregated_df.show()

# Step 2: Prepare Features
# Encode business_id
indexer_business = StringIndexer(inputCol='business_id', outputCol='business_id_indexed')
indexer_business_model = indexer_business.fit(aggregated_df)  # Fit the indexer
aggregated_df = indexer_business_model.transform(aggregated_df)

# Encode sentiment and topic
indexer_sentiment = StringIndexer(inputCol='sentiment', outputCol='sentiment_indexed')
indexer_sentiment_model = indexer_sentiment.fit(aggregated_df)  # Fit the indexer
aggregated_df = indexer_sentiment_model.transform(aggregated_df)

indexer_topic = StringIndexer(inputCol='topic', outputCol='topic_indexed')
indexer_topic_model = indexer_topic.fit(aggregated_df)  # Fit the indexer
aggregated_df = indexer_topic_model.transform(aggregated_df)

# Step 3: Create feature vectors
vector_assembler = VectorAssembler(inputCols=['business_id_indexed'], outputCol='features')
aggregated_df = vector_assembler.transform(aggregated_df)

# Features and target variables
X = aggregated_df.select('features')
y_sentiment = aggregated_df.select('sentiment_indexed')
y_topic = aggregated_df.select('topic_indexed')

# Step 4: Train Models
# Split the data for sentiment prediction
train_sentiment, test_sentiment = aggregated_df.randomSplit([0.8, 0.2], seed=42)

# Train sentiment model using Logistic Regression
sentiment_model = LogisticRegression(featuresCol='features', labelCol='sentiment_indexed')
sentiment_model = sentiment_model.fit(train_sentiment)

# Evaluate sentiment model
predictions_sentiment = sentiment_model.transform(test_sentiment)
evaluator_sentiment = MulticlassClassificationEvaluator(labelCol='sentiment_indexed', predictionCol='prediction', metricName='accuracy')
accuracy_sentiment = evaluator_sentiment.evaluate(predictions_sentiment)
print(f"Sentiment Model Accuracy: {accuracy_sentiment}")

# Split the data for topic prediction
train_topic, test_topic = aggregated_df.randomSplit([0.8, 0.2], seed=42)

# Train topic model using Logistic Regression
topic_model = LogisticRegression(featuresCol='features', labelCol='topic_indexed')
topic_model = topic_model.fit(train_topic)

# Evaluate topic model
predictions_topic = topic_model.transform(test_topic)
evaluator_topic = MulticlassClassificationEvaluator(labelCol='topic_indexed', predictionCol='prediction', metricName='accuracy')
accuracy_topic = evaluator_topic.evaluate(predictions_topic)
print(f"Topic Model Accuracy: {accuracy_topic}")

# Save the indexers and models for future predictions
indexer_business_model.save("/content/drive/MyDrive/JTG_TEST/indexer_business")
indexer_sentiment_model.save("/content/drive/MyDrive/JTG_TEST/indexer_sentiment")
indexer_topic_model.save("/content/drive/MyDrive/JTG_TEST/indexer_topic")
sentiment_model.save("/content/drive/MyDrive/JTG_TEST/sentiment_model")
topic_model.save("/content/drive/MyDrive/JTG_TEST/topic_model")

+--------------------+---------+----------------+
|         business_id|sentiment|           topic|
+--------------------+---------+----------------+
|---kPU91CF4Lq2-Wl...| positive|    food_quality|
|--qLiYw2ErSmvVwum...| positive|    food_quality|
|-0eUa8TsXFFy0FCxH...| positive|    food_quality|
|-1XSzguS6XLN-V6MV...| positive|    food_quality|
|-1owBLC2h6DF5n_j7...| negative|customer_service|
|-2gDGXUYPigcPbMID...| positive|    food_quality|
|-3Xl8nSBSjaPpftsS...| negative|customer_service|
|-4qgeqxb2hKe3dUUH...| positive|    food_quality|
|-6OjnX3ZdDOhHxWR6...| positive|    food_quality|
|-7ZTpN4KmfWotnu4r...| negative|    food_quality|
|-7wXZWnKe26iwJY_5...| positive|    food_quality|
|-81BXpO5Fuk-RqCab...| positive|    food_quality|
|-9rRsa_9GTkSYkPdK...| positive|    food_quality|
|-B7E8MFb47FoGcp25...| positive|    food_quality|
|-BjXcRcvDqOQ23eUK...| positive|customer_service|
|-D_InZvA0lq497Rto...| positive|    food_quality|
|-DcAq3aOYlMTM0vzO...| positive|    food_quality|


In [45]:
# # Save the sentiment model
# sentiment_model.save("/content/drive/MyDrive/JTG_TEST/sentiment_model")

# # Save the topic model
# topic_model.save("/content/drive/MyDrive/JTG_TEST/topic_model")

**Trying Prediction**

In [47]:
# from pyspark.sql import SparkSession
# from pyspark.ml.feature import StringIndexerModel
# from pyspark.ml.feature import VectorAssembler
# from pyspark.ml.classification import LogisticRegressionModel

# # Initialize Spark session
# spark = SparkSession.builder.appName("Sentiment and Topic Prediction").getOrCreate()

# # Load the indexers and models
# indexer_business_model = StringIndexerModel.load("/content/drive/MyDrive/JTG_TEST/indexer_business")
# indexer_sentiment_model = StringIndexerModel.load("/content/drive/MyDrive/JTG_TEST/indexer_sentiment")
# indexer_topic_model = StringIndexerModel.load("/content/drive/MyDrive/JTG_TEST/indexer_topic")
# sentiment_model = LogisticRegressionModel.load("/content/drive/MyDrive/JTG_TEST/sentiment_model")
# topic_model = LogisticRegressionModel.load("/content/drive/MyDrive/JTG_TEST/topic_model")

# def predict_sentiment_and_topic(business_id):
#     # Create a DataFrame with the input business ID
#     input_data = spark.createDataFrame([(business_id,)], ['business_id'])

#     # Encode the business ID using the trained indexer
#     input_data = indexer_business_model.transform(input_data)

#     # Create a feature vector
#     vector_assembler = VectorAssembler(inputCols=['business_id_indexed'], outputCol='features')
#     input_data = vector_assembler.transform(input_data)

#     # Make predictions using the sentiment model
#     sentiment_prediction = sentiment_model.transform(input_data)
#     predicted_sentiment = sentiment_prediction.select('prediction').collect()[0][0]

#     # Make predictions using the topic model
#     topic_prediction = topic_model.transform(input_data)
#     predicted_topic = topic_prediction.select('prediction').collect()[0][0]

#     return predicted_sentiment, predicted_topic

# # Example usage
# business_id = 'TdyHRkxn334JxtUSdGpIiw'  # Replace with your actual business ID
# predicted_sentiment, predicted_topic = predict_sentiment_and_topic(business_id)
# print(f"Predicted sentiment: {predicted_sentiment}, Predicted topic: {predicted_topic}")

Predicted sentiment: 0.0, Predicted topic: 0.0


Debugging

In [18]:
def predict_sentiment_and_topic(business_id):
    # Create a DataFrame with the input business ID
    input_data = spark.createDataFrame([(business_id,)], ['business_id'])

    # Encode the business ID using the trained indexer
    input_data = indexer_business_model.transform(input_data)
    print("After Indexing:")
    input_data.show()

    # Create a feature vector
    vector_assembler = VectorAssembler(inputCols=['business_id_indexed'], outputCol='features')
    input_data = vector_assembler.transform(input_data)
    print("After Vector Assembling:")
    input_data.show()

    # Make predictions using the sentiment model
    sentiment_prediction = sentiment_model.transform(input_data)
    print("Sentiment Prediction:")
    sentiment_prediction.show()

    # # Make predictions using the topic model
    # topic_prediction = topic_model.transform(input_data)
    # print("Topic Prediction:")
    # topic_prediction.show()

    # Extract predicted values
    predicted_sentiment = sentiment_prediction.select('prediction').collect()[0][0]
    # predicted_topic = topic_prediction.select('prediction').collect()[0][0]

    return predicted_sentiment#, predicted_topic

In [21]:
business_id = 'hiP3H0wW45I0jZI7KafzZg'  # Replace with your actual business ID
predicted_sentiment, predicted_topic = predict_sentiment_and_topic(business_id)
print(f"Predicted sentiment: {predicted_sentiment}, Predicted topic: {predicted_topic}")

After Indexing:
+--------------------+-------------------+
|         business_id|business_id_indexed|
+--------------------+-------------------+
|hiP3H0wW45I0jZI7K...|            22803.0|
+--------------------+-------------------+

After Vector Assembling:
+--------------------+-------------------+---------+
|         business_id|business_id_indexed| features|
+--------------------+-------------------+---------+
|hiP3H0wW45I0jZI7K...|            22803.0|[22803.0]|
+--------------------+-------------------+---------+

Sentiment Prediction:
+--------------------+-------------------+---------+--------------------+--------------------+----------+
|         business_id|business_id_indexed| features|       rawPrediction|         probability|prediction|
+--------------------+-------------------+---------+--------------------+--------------------+----------+
|hiP3H0wW45I0jZI7K...|            22803.0|[22803.0]|[2.02216052609841...|[0.88310422739244...|       0.0|
+--------------------+---------

In [53]:
# Get the mapping of indices to labels
sentiment_mapping = indexer_sentiment_model.labels
print(sentiment_mapping)  # Output: ['negative', 'neutral', 'positive']

['positive', 'negative']


In [54]:
topic_mapping = indexer_topic_model.labels
print(topic_mapping)

['customer_service', 'food_quality', 'ambience', 'price']


In [ ]:
# 0.0 corresponds to customer_service
# 1.0 corresponds to food_quality
# 2.0 corresponds to ambience
# 3.0 corresponds to price

# sentiment_indexer
# 0 positive
# 1 negative


AttributeError: 'StringIndexer' object has no attribute 'transform'

In [ ]:
# Step 4: Make Predictions
def predict_sentiment_and_topic(business_id):
    # Transform business_id to indexed value
    business_id_indexed = indexer.transform(spark.createDataFrame([(business_id,)], ['business_id'])).collect()[0][1]
    features = spark.createDataFrame([(business_id_indexed,)], ['business_id_indexed'])

    predicted_sentiment = sentiment_model.transform(features).select('prediction').collect()[0][0]
    predicted_topic = topic_model.transform(features).select('prediction').collect()[0][0]

    return predicted_sentiment, predicted_topic

# Example prediction
predicted_sentiment, predicted_topic = predict_sentiment_and_topic('b1')
print(f"\nPredicted Sentiment for business_id 'b1': {predicted_sentiment}")
print(f"Predicted Topic for business_id 'b1': {predicted_topic}")

In [56]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, when
from pyspark.sql.types import StringType
from pyspark.ml.feature import Tokenizer, HashingTF, IDF, StringIndexer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# # Select relevant columns and preprocess
# preprocessed_df = review_df.select("business_id", "text")
# preprocessed_df = preprocessed_df.withColumn("Review", preprocess_udf(col("text")))

# # Add labels for sentiment and topic
# preprocessed_df = preprocessed_df.withColumn(
#     "sentiment",
#     when(col("Review").rlike("(?i)good|great|excellent"), "positive").otherwise("negative")
# )

# preprocessed_df = preprocessed_df.withColumn(
#     "topic",
#     when(col("Review").rlike("(?i)service"), "customer_service").otherwise("food_quality")
# )



In [57]:
!wget -q https://dlcdn.apache.org/spark/spark-3.5.3/spark-3.5.3-bin-hadoop3.tgz


In [58]:
!file spark-3.5.3-bin-hadoop3.tgz


spark-3.5.3-bin-hadoop3.tgz: gzip compressed data, from Unix, original size modulo 2^32 445972480


In [59]:
!tar -xvzf spark-3.5.3-bin-hadoop3.tgz


spark-3.5.3-bin-hadoop3/
spark-3.5.3-bin-hadoop3/data/
spark-3.5.3-bin-hadoop3/data/graphx/
spark-3.5.3-bin-hadoop3/data/graphx/users.txt
spark-3.5.3-bin-hadoop3/data/graphx/followers.txt
spark-3.5.3-bin-hadoop3/data/mllib/
spark-3.5.3-bin-hadoop3/data/mllib/sample_linear_regression_data.txt
spark-3.5.3-bin-hadoop3/data/mllib/sample_fpgrowth.txt
spark-3.5.3-bin-hadoop3/data/mllib/sample_libsvm_data.txt
spark-3.5.3-bin-hadoop3/data/mllib/gmm_data.txt
spark-3.5.3-bin-hadoop3/data/mllib/kmeans_data.txt
spark-3.5.3-bin-hadoop3/data/mllib/streaming_kmeans_data_test.txt
spark-3.5.3-bin-hadoop3/data/mllib/sample_lda_data.txt
spark-3.5.3-bin-hadoop3/data/mllib/sample_multiclass_classification_data.txt
spark-3.5.3-bin-hadoop3/data/mllib/pagerank_data.txt
spark-3.5.3-bin-hadoop3/data/mllib/sample_isotonic_regression_libsvm_data.txt
spark-3.5.3-bin-hadoop3/data/mllib/sample_lda_libsvm_data.txt
spark-3.5.3-bin-hadoop3/data/mllib/sample_movielens_data.txt
spark-3.5.3-bin-hadoop3/data/mllib/pic_data

In [60]:
import os
if os.path.exists("spark-3.5.3-bin-hadoop3.tgz"):  # Corrected to match the actual downloaded file name
    !tar -xvzf spark-3.5.3-bin-hadoop3.tgz  # Extract the tarball
else:
    print("Download failed or file not found. Please check the URL or try again.")

# Step 5: Install findspark
!pip install -q findspark

# Step 6: Set environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.3-bin-hadoop3"  # Corrected to match the extracted folder name

# Step 7: Initialize findspark
import findspark
findspark.init()

spark-3.5.3-bin-hadoop3/
spark-3.5.3-bin-hadoop3/data/
spark-3.5.3-bin-hadoop3/data/graphx/
spark-3.5.3-bin-hadoop3/data/graphx/users.txt
spark-3.5.3-bin-hadoop3/data/graphx/followers.txt
spark-3.5.3-bin-hadoop3/data/mllib/
spark-3.5.3-bin-hadoop3/data/mllib/sample_linear_regression_data.txt
spark-3.5.3-bin-hadoop3/data/mllib/sample_fpgrowth.txt
spark-3.5.3-bin-hadoop3/data/mllib/sample_libsvm_data.txt
spark-3.5.3-bin-hadoop3/data/mllib/gmm_data.txt
spark-3.5.3-bin-hadoop3/data/mllib/kmeans_data.txt
spark-3.5.3-bin-hadoop3/data/mllib/streaming_kmeans_data_test.txt
spark-3.5.3-bin-hadoop3/data/mllib/sample_lda_data.txt
spark-3.5.3-bin-hadoop3/data/mllib/sample_multiclass_classification_data.txt
spark-3.5.3-bin-hadoop3/data/mllib/pagerank_data.txt
spark-3.5.3-bin-hadoop3/data/mllib/sample_isotonic_regression_libsvm_data.txt
spark-3.5.3-bin-hadoop3/data/mllib/sample_lda_libsvm_data.txt
spark-3.5.3-bin-hadoop3/data/mllib/sample_movielens_data.txt
spark-3.5.3-bin-hadoop3/data/mllib/pic_data

In [61]:
import os
print("SPARK_HOME:", os.environ.get("SPARK_HOME"))
print("JAVA_HOME:", os.environ.get("JAVA_HOME"))


SPARK_HOME: /content/spark-3.5.3-bin-hadoop3
JAVA_HOME: /usr/lib/jvm/java-8-openjdk-amd64


In [62]:
!pip uninstall pyspark -y
!pip install pyspark


Found existing installation: pyspark 3.5.3
Uninstalling pyspark-3.5.3:
  Successfully uninstalled pyspark-3.5.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=efb47c32cbc8d16e7710bbfea729f3ba55076be1b44f900761031b7ac15686e0
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [27]:
import os

# Set environment variables for Java and Spark
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.3-bin-hadoop3"  # Make sure this path is correct

# Create Spark session with minimal configurations
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Spark Setup Verification") \
    .getOrCreate()  # Try without additional config options first

# Print Spark version to verify setup
print(f"Spark version: {spark.version}")

# Stop the session
spark.stop()


Spark version: 3.5.3


In [63]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.0-bin-hadoop3"


In [64]:
import findspark
findspark.init()


In [65]:
# Import required libraries
import time
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType
from pyspark.ml.feature import Tokenizer, HashingTF, IDF, StringIndexer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline

# Import NLTK for preprocessing
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
"""
# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Text Classification with Progress") \
    .getOrCreate()

# Define a function to log progress
def log_progress(stage, start_time, current_step, total_steps):
    elapsed_time = time.time() - start_time
    percentage = (current_step / total_steps) * 100
    print(f"[{percentage:.2f}%] {stage} completed in {elapsed_time:.2f} seconds.")

# Measure overall execution time
overall_start_time = time.time()

# Stage 1: Preprocessing function
start_time = time.time()
def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalpha() and word not in stop_words]
    return ' '.join(tokens)

preprocess_udf = udf(preprocess_text, StringType())
log_progress("Preprocessing function setup", start_time, 1, 6)

# Stage 2: Create training dataset
start_time = time.time()
data = [
    ("1", "The food was great and the service was excellent", "positive", "food_quality"),
    ("2", "Terrible service, but the food was good", "negative", "customer_service"),
    ("3", "Amazing service, but the food was cold", "positive", "customer_service"),
    ("4", "The food was terrible and the service was awful", "negative", "food_quality"),
]
columns = ["business_id", "text", "sentiment", "topic"]
train_df = spark.createDataFrame(data, columns)
preprocessed_df = train_df.withColumn("Review", preprocess_udf(col("text")))
log_progress("Training dataset creation and preprocessing", start_time, 2, 6)

# Stage 3: Define the pipeline stages
start_time = time.time()
tokenizer = Tokenizer(inputCol="Review", outputCol="words")
hashing_tf = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=1000)
idf = IDF(inputCol="rawFeatures", outputCol="features")
sentiment_indexer = StringIndexer(inputCol="sentiment", outputCol="sentimentIndex")
rf_sentiment = RandomForestClassifier(labelCol="sentimentIndex", featuresCol="features", numTrees=50, maxDepth=5)
pipeline = Pipeline(stages=[tokenizer, hashing_tf, idf, sentiment_indexer, rf_sentiment])
log_progress("Pipeline setup", start_time, 3, 6)

# Stage 4: Fit the pipeline
start_time = time.time()
model = pipeline.fit(preprocessed_df)
log_progress("Pipeline fitting", start_time, 4, 6)

# Stage 5: Prepare the test dataset
start_time = time.time()
test_data = [
    ("5", "Great service and delicious food"),
    ("6", "Awful service but tasty meals"),
    ("7", "Terrible food and bad service"),
    ("8", "Amazing meals with wonderful service"),
]
test_columns = ["business_id", "text"]
test_df = spark.createDataFrame(test_data, test_columns)
test_df = test_df.withColumn("Review", preprocess_udf(col("text")))
log_progress("Test dataset preparation", start_time, 5, 6)

# Stage 6: Generate predictions
start_time = time.time()
predictions = model.transform(test_df)
log_progress("Generating predictions", start_time, 6, 6)

# Display predictions
predictions.select("business_id", "Review", "prediction").show(truncate=False)

# Log overall execution time
overall_elapsed_time = time.time() - overall_start_time
print(f"Pipeline completed in {overall_elapsed_time:.2f} seconds.")
"""

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


'\n# Initialize SparkSession\nspark = SparkSession.builder     .appName("Text Classification with Progress")     .getOrCreate()\n\n# Define a function to log progress\ndef log_progress(stage, start_time, current_step, total_steps):\n    elapsed_time = time.time() - start_time\n    percentage = (current_step / total_steps) * 100\n    print(f"[{percentage:.2f}%] {stage} completed in {elapsed_time:.2f} seconds.")\n\n# Measure overall execution time\noverall_start_time = time.time()\n\n# Stage 1: Preprocessing function\nstart_time = time.time()\ndef preprocess_text(text):\n    stop_words = set(stopwords.words(\'english\'))\n    lemmatizer = WordNetLemmatizer()\n    tokens = word_tokenize(text.lower())\n    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalpha() and word not in stop_words]\n    return \' \'.join(tokens)\n\npreprocess_udf = udf(preprocess_text, StringType())\nlog_progress("Preprocessing function setup", start_time, 1, 6)\n\n# Stage 2: Create training datase

In [66]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Download required NLTK resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Initialize objects outside the UDF function
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Define the preprocessing function
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalpha() and word not in stop_words]
    return ' '.join(tokens)

# Convert the Python function to a UDF
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

preprocess_udf = udf(preprocess_text, StringType())


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [67]:
# Initialize stopwords and lemmatizer outside the UDF
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Define the preprocessing function
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalpha() and word not in stop_words]
    return ' '.join(tokens)

# Convert the Python function to a UDF
preprocess_udf = udf(preprocess_text, StringType())

# Create the DataFrame with the preprocessed text
train_df = spark.createDataFrame(data, columns)
preprocessed_df = train_df.withColumn("Review", preprocess_udf(col("text")))

# Now continue with the rest of the pipeline setup


AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `text` cannot be resolved. Did you mean one of the following? [`id`, `Review`].;
'Project [id#7231, preprocess_text('text)#7235 AS Review#7236]
+- LogicalRDD [id#7231, Review#7232], false


In [ ]:
from pyspark.sql.functions import pandas_udf
import pandas as pd

# Define a Pandas UDF
@pandas_udf(StringType())
def preprocess_text_pandas_udf(text: pd.Series) -> pd.Series:
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    return text.apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in word_tokenize(x.lower()) if word.isalpha() and word not in stop_words]))

# Use Pandas UDF in the DataFrame
preprocessed_df = train_df.withColumn("Review", preprocess_text_pandas_udf(col("text")))


In [32]:
from pyspark.sql import SparkSession
# Split into training and testing datasets
train_df, test_df = preprocessed_df.randomSplit([0.8, 0.2], seed=42)

# Tokenize and vectorize
tokenizer = Tokenizer(inputCol="Review", outputCol="words")
hashing_tf = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=5000)
idf = IDF(inputCol="rawFeatures", outputCol="features")

# Encode labels
sentiment_indexer = StringIndexer(inputCol="sentiment", outputCol="sentimentIndex")
topic_indexer = StringIndexer(inputCol="topic", outputCol="topicIndex")



AssertionError: 

In [ ]:

"""
sampled_train_df = train_df.sample(fraction=0.1, seed=42)  # 10% of the data
model = pipeline.fit(sampled_train_df)
"""


In [ ]:
# Classifiers with custom column names for probability and prediction columns
rf_sentiment = RandomForestClassifier(
    labelCol="sentimentIndex",
    featuresCol="features",
    rawPredictionCol="rawPredictionSentiment",  # Custom name for raw prediction
    predictionCol="sentimentPrediction",  # Custom name for prediction column
    probabilityCol="probabilitySentiment"  # Custom name for probability column
)

rf_topic = RandomForestClassifier(
    labelCol="topicIndex",
    featuresCol="features",
    rawPredictionCol="rawPredictionTopic",  # Custom name for raw prediction
    predictionCol="topicPrediction",  # Custom name for prediction column
    probabilityCol="probabilityTopic"  # Custom name for probability column
)

# Pipeline
pipeline = Pipeline(stages=[tokenizer, hashing_tf, idf, sentiment_indexer, topic_indexer, rf_sentiment, rf_topic])

# Train the model
model = pipeline.fit(train_df)


In [ ]:
# Make predictions
predictions = model.transform(test_df)

# Display results
predictions.select("Review", "sentiment", "topic", "sentimentPrediction", "topicPrediction").show(truncate=False)


+----------------------------+---------+------------+-------------------+---------------+
|Review                      |sentiment|topic       |sentimentPrediction|topicPrediction|
+----------------------------+---------+------------+-------------------+---------------+
|food great service excellent|positive |food_quality|0.0                |0.0            |
+----------------------------+---------+------------+-------------------+---------------+



In [37]:
# prompt: i have the preprocessed df now, i need to split it io training an test and then train a random forest model to analyse the review
# spark.stop()

spark = SparkSession.builder.appName("SentimentAnalysis").getOrCreate()
from pyspark.ml.feature import Tokenizer, HashingTF, IDF, StringIndexer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.sql.functions import col, when, lit, pandas_udf
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
import pandas as pd
import time

# ... (your existing code)

# Split into training and testing datasets
train_df, test_df = preprocessed_df.randomSplit([0.8, 0.2], seed=42)

# Tokenize and vectorize
tokenizer = Tokenizer(inputCol="Review", outputCol="words")
hashing_tf = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=5000)
idf = IDF(inputCol="rawFeatures", outputCol="features")

# Encode labels
sentiment_indexer = StringIndexer(inputCol="sentiment", outputCol="sentimentIndex")
topic_indexer = StringIndexer(inputCol="topic", outputCol="topicIndex")

# Classifiers with custom column names for probability and prediction columns
rf_sentiment = RandomForestClassifier(
    labelCol="sentimentIndex",
    featuresCol="features",
    rawPredictionCol="rawPredictionSentiment",  # Custom name for raw prediction
    predictionCol="sentimentPrediction",  # Custom name for prediction column
    probabilityCol="probabilitySentiment"  # Custom name for probability column
)

rf_topic = RandomForestClassifier(
    labelCol="topicIndex",
    featuresCol="features",
    rawPredictionCol="rawPredictionTopic",  # Custom name for raw prediction
    predictionCol="topicPrediction",  # Custom name for prediction column
    probabilityCol="probabilityTopic"  # Custom name for probability column
)

# Pipeline
pipeline = Pipeline(stages=[tokenizer, hashing_tf, idf, sentiment_indexer, topic_indexer, rf_sentiment, rf_topic])

# Train the model
model = pipeline.fit(train_df)

# Make predictions
predictions = model.transform(test_df)

# Display results
predictions.select("Review", "sentiment", "topic", "sentimentPrediction", "topicPrediction").show(truncate=False)

Py4JJavaError: An error occurred while calling o754.fit.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4j.ClientServerConnection.wait

In [8]:
# prompt: select only the needed code from this notebook and create an end to end code which each step in the right orde

from google.colab import drive
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf, col, when, lit, concat_ws, collect_list
from pyspark.ml.feature import Tokenizer, HashingTF, IDF, StringIndexer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.sql.functions import pandas_udf
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import os

# Mount Google Drive
drive.mount('/content/drive')

# Install necessary libraries
!pip install pyspark nltk scikit-learn

# Download NLTK resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')

# Initialize SparkSession
spark = SparkSession.builder.appName("SentimentAnalysis").getOrCreate()

# Load the CSV file into a DataFrame
review_df = spark.read.csv("/content/drive/MyDrive/ProcessedCSV/Recommender_System_Newdata.csv", header=True, inferSchema=True)

# Data Cleaning (remove rows with values matching the first row)
columns_to_drop = review_df.columns
review_df = review_df.filter(F.col(columns_to_drop[0]) != review_df.first()[columns_to_drop[0]])
for col in columns_to_drop[1:]:
    review_df = review_df.filter(F.col(col) != review_df.first()[col])

# Select relevant columns and group/concatenate text
selected_df = review_df.select("business_id", "text")
concatenated_df = selected_df.groupBy("business_id").agg(concat_ws(" ", collect_list("text")).alias("concatenated_text"))

# Preprocessing function
# Initialize stopwords and lemmatizer outside the UDF
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalpha() and word not in stop_words]
    return ' '.join(tokens)

# Convert the Python function to a Pandas UDF
@pandas_udf(StringType())
def preprocess_text_pandas_udf(text: pd.Series) -> pd.Series:
    return text.apply(preprocess_text)

# Use Pandas UDF in the DataFrame
preprocessed_df = concatenated_df.withColumn("Review", preprocess_text_pandas_udf(F.col("concatenated_text")))

# Add labels for sentiment and topic
preprocessed_df = preprocessed_df.withColumn(
    "sentiment",
    when(F.col("Review").rlike("(?i)good|great|excellent|incredible|fantastic|awesome|amaz"), "positive").otherwise("negative")
)

topic_keywords = {
    "customer_service": ["service", "staff", "waiter", "waitress", "server"],
    "food_quality": ["food", "taste", "delicious", "flavor", "meal"],
    "ambience": ["atmosphere", "ambiance", "decor", "music", "lighting"],
    "price": ["price", "expensive", "cheap", "cost", "value"]
}
conditions = []
for topic, keywords in topic_keywords.items():
    for keyword in keywords:
      conditions.append(when(F.col("Review").rlike(f"(?i){keyword}"), topic))

preprocessed_df = preprocessed_df.withColumn(
    "topic",
    F.coalesce(*conditions, lit("other"))
)


# Split into training and testing datasets
train_df, test_df = preprocessed_df.randomSplit([0.8, 0.2], seed=42)

# Define the ML pipeline
tokenizer = Tokenizer(inputCol="Review", outputCol="words")
hashing_tf = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=5000)
idf = IDF(inputCol="rawFeatures", outputCol="features")
sentiment_indexer = StringIndexer(inputCol="sentiment", outputCol="sentimentIndex")
topic_indexer = StringIndexer(inputCol="topic", outputCol="topicIndex")

rf_sentiment = RandomForestClassifier(labelCol="sentimentIndex", featuresCol="features", predictionCol="sentimentPrediction", probabilityCol="probabilitySentiment", rawPredictionCol="rawPredictionSentiment")
rf_topic = RandomForestClassifier(labelCol="topicIndex", featuresCol="features", predictionCol="topicPrediction", probabilityCol="probabilityTopic", rawPredictionCol="rawPredictionTopic")


pipeline = Pipeline(stages=[tokenizer, hashing_tf, idf, sentiment_indexer, topic_indexer, rf_sentiment, rf_topic])

# Train the model
model = pipeline.fit(train_df)

# Make predictions
predictions = model.transform(test_df)

# Show the predictions
predictions.select("business_id", "Review", "sentiment", "topic", "sentimentPrediction", "topicPrediction").show(truncate=False)

# spark.stop()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [7]:
# prompt: how to make a prediction for a business id
spark = SparkSession.builder.appName("SentimentAnalysis").getOrCreate()

from pyspark.sql.functions import lit, when, col, concat_ws, collect_list
from pyspark.ml.feature import Tokenizer, HashingTF, IDF, StringIndexer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import StringType
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Assuming 'preprocessed_df' and 'spark' are already defined from the previous code

# ... (your existing code)

# Function to predict for a single business ID
def predict_for_business(business_id, model, preprocessed_df):
    # Filter for the specific business ID
    business_data = preprocessed_df.filter(col("business_id") == business_id)

    if business_data.count() == 0:
      print(f"No data found for business ID: {business_id}")
      return None

    # Make predictions using the trained model
    predictions = model.transform(business_data)

    # Select relevant columns from predictions
    return predictions.select("business_id", "Review", "sentiment", "topic", "sentimentPrediction", "topicPrediction").first()



# Example usage:
business_id_to_predict = "-LmhsdQproqCf5EQoD06rQ"  # Replace with the actual business ID
prediction_result = predict_for_business(business_id_to_predict, model, preprocessed_df)

if prediction_result:
    print(f"Prediction for business ID {business_id_to_predict}:")
prediction_result

Py4JJavaError: An error occurred while calling o1349.count.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4j.ClientServerConnection.w

In [11]:
# prompt: read and display /content/drive/MyDrive/ProcessedCSV/Recommender_System_Newdata.csv

import pandas as pd

# Load the CSV file into a pandas DataFrame
df = spark.read.csv('/content/drive/MyDrive/ProcessedCSV/Recommender_System_Newdata.csv', header = True)

# Display the DataFrame
df.show()

+--------------------+--------------------+--------------------+--------------------+--------------+-----+-----------+-------------+---------------+------+------------+--------------------+-------+----+------------+
|         business_id|            rev_text|            bus_name|             address|          city|state|postal_code|     latitude|      longitude|rating|review_count|            tip_text|parking|WiFi|cuisine_type|
+--------------------+--------------------+--------------------+--------------------+--------------+-----+-----------+-------------+---------------+------+------------+--------------------+-------+----+------------+
|zljfjg_zTL3ceGi_V...|It's ok if you li...|Sheraton New Orle...|        500 Canal St|   New Orleans|   LA|      70130|29.9521458122| -90.0681066513|   3.0|         673|                NULL|    Yes| Yes|      Creole|
|3Y2PxEwN2iNHAQj4Z...|"Love their smoot...|       Smoothie King|    1413 S Hanley Rd|     Brentwood|   MO|      63144|    38.627224| -90